In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
import warnings

import pandas_ta as ta
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_pickle("train_기술적분석_다중공선성처리완료.pkl")
test = pd.read_pickle("prediction_기술적분석_다중공선성처리완료.pkl")

In [6]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
from tqdm import tqdm
from joblib import Parallel, delayed

# XGBoost 하이퍼파라미터의 탐색 공간 정의
# param_grid = {
#     'n_estimators': [100, 200, 500, 1000],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
#     'min_child_weight': [1, 3, 5],
#     'subsample': [0.5, 0.7, 1],
#     'colsample_bytree': [0.5, 0.7, 1]
# }

param_grid = {
    'n_estimators': [100],
    'max_depth': [3],
    'learning_rate': [0.01],
}


results_df = pd.DataFrame(columns=['ticker', 'final_return'])

# 특정 주식에 대해 Grid Search를 수행하는 함수
def grid_search_for_one_stock(ticker, task:pd.DataFrame):
    # 특정 주식 데이터만 선택
    data = train[train['ticker'] == ticker]
    task = test[test['ticker'] == ticker]

    # 특성 및 타겟 정의
    X = data.drop(['ticker', 'firm', 'target'], axis=1)
    y = data['target']

    X_pred = task.drop(['ticker', 'firm', 'target'], axis=1)

    # XGBoost 모델 생성
    model = XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist')
    model.fit(X, y)

    # Grid Search 실행
    #tscv = TimeSeriesSplit(n_splits=3)
    #grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='neg_mean_squared_error')
    #grid_search.fit(X, y)

    # 최적 하이퍼파라미터를 사용하여 학습
    #best_model = grid_search.best_estimator_
    #best_model.fit(X, y)

    # 향후 15개의 거래일에 대해서 예측
    predictions = model.predict(X_pred)

    # 최종 수익률 계산
    final_return = (predictions[-1] - predictions[0]) / predictions[0]

    # 결과 저장
    return ticker, final_return

# 모든 주식에 대해 병렬로 Grid Search 실행
results = Parallel(n_jobs=-1)(delayed(grid_search_for_one_stock)(ticker, test) for ticker in train['ticker'].unique())

In [7]:
answer = pd.DataFrame(results, columns=['ticker', 'return'])
answer.head()

,ticker,return
0,A000020,-0.009876
1,A000040,0.024674
2,A000050,-0.024016
3,A000070,-0.003829
4,A000080,0.009516


In [16]:
answer = pd.read_csv("./sub/model01_281724.csv")

In [17]:
sample = pd.read_csv("./sub/sample_submission.csv")
sample = sample.rename(columns = {'종목코드' : 'ticker', '순위' : 'rating'})

# sample의 'ticker' 컬럼에서 answer의 'ticker' 컬럼에 없는 값을 선택
missing_tickers = sample[~sample['ticker'].isin(answer['ticker'])]
missing_tickers = missing_tickers.reset_index(drop=True)
missing_tickers = missing_tickers.drop('rating', axis=1)
missing_tickers['return'] = 0
missing_tickers.head()

submission = pd.merge(answer, missing_tickers, how='outer')
submission = submission.sort_values('return', ascending=False)
submission = submission.reset_index()
submission = submission.reset_index()
submission = submission.drop('index', axis=1)
submission = submission.sort_values('ticker')
submission['순위2'] = submission['level_0']
submission = submission.drop(['level_0', 'return'], axis=1)
submission = submission.reset_index(drop=True)
submission['순위'] = submission['순위2'] + 1
submission = submission.rename(columns = {'ticker' : '종목코드'})
submission = submission.drop('순위2', axis=1)

submission.to_csv("XGBoost5.csv")

In [19]:
submission = pd.read_csv("./sub/XGBoost5.csv")
len(submission['순위'].unique())

1957